In [1]:
from PIL import Image
import os
# https://pillow.readthedocs.io/en/stable/handbook/tutorial.html

In [2]:
ROOT_OF_IMAGES = './ppcovers/'
REGION_FOLDERS = ["africa","oceania","asia","south_america","north_america","europe"]


region_to_full_pp_cover_paths = {}
for region_name in REGION_FOLDERS:
  region_folder_path = os.path.join(ROOT_OF_IMAGES,region_name)
  pp_images_in_folder = os.listdir(region_folder_path)
  full_image_paths_for_region = [os.path.join(region_folder_path,pp_image_filename) for pp_image_filename in pp_images_in_folder]
  region_to_full_pp_cover_paths[region_name] = full_image_paths_for_region
region_to_full_pp_cover_paths


{'africa': ['./ppcovers/africa/Kenya.png',
  './ppcovers/africa/Ghana.png',
  './ppcovers/africa/Liberia.png',
  './ppcovers/africa/Sao+Tome+and+Principe.png',
  './ppcovers/africa/Namibia.png',
  './ppcovers/africa/Tanzania.png',
  './ppcovers/africa/Zambia.png',
  './ppcovers/africa/Djibouti.png',
  './ppcovers/africa/Angola.png',
  './ppcovers/africa/Lesotho.png',
  './ppcovers/africa/Tunisia.png',
  './ppcovers/africa/Sierra+Leone.png',
  './ppcovers/africa/Guinea.png',
  './ppcovers/africa/Algeria.png',
  './ppcovers/africa/Rwanda.png',
  './ppcovers/africa/Cameroon.png',
  './ppcovers/africa/Equatorial+Guinea.png',
  './ppcovers/africa/Burundi.png',
  './ppcovers/africa/Seychelles.png',
  './ppcovers/africa/Mozambique.png',
  './ppcovers/africa/Benin.png',
  './ppcovers/africa/Gabon.png',
  './ppcovers/africa/Libya.png',
  './ppcovers/africa/Cape+Verde.png',
  './ppcovers/africa/South+Sudan.png',
  './ppcovers/africa/Madagascar.png',
  './ppcovers/africa/Senegal.png',
  './ppcove

In [3]:
from collections import defaultdict

image_sizes = defaultdict(lambda :  0)
image_aspect_ratios = defaultdict(lambda :  0)
for region in region_to_full_pp_cover_paths:
  for image_path in region_to_full_pp_cover_paths[region]:
    image = Image.open(image_path)
    image_sizes[image.size] += 1
    image_aspect_ratios[image.size] += 1


for image_size in image_sizes:
  print(f"{image_size}:{image_sizes[image_size]}")
# print(image_aspect_ratios)
print(f"min image size: {min([size[0] for size in image_sizes.keys()])} ")


(667, 1000):32
(674, 1000):68
(705, 1000):54
(327, 485):8
(1056, 1500):1
(598, 850):1
(323, 485):2
(308, 448):2
(707, 1000):17
(723, 1000):1
(719, 1000):7
(718, 1000):1
(710, 1000):1
(561, 764):1
(696, 1000):1
(667, 971):1
(1087, 1542):1
min image size: 308 


The region is defined by a 4-tuple, where coordinates are (left, upper, right, lower). The Python Imaging Library uses a coordinate system with (0, 0) in the upper left corner. Also note that coordinates refer to positions between the pixels, so the region in the above example is exactly 300x300 pixels.

In [4]:
# box should be (667 wide, and 10000 tall)

def get_box_for_image_path(image_path):
  desired_box_width = 667
  desired_box_height = 1000   # 2/3 aspect ratio
  image = Image.open(image_path)
  (image_width, image_height) = image.size
  extra_width_pixels = image_width - desired_box_width
  extra_height_pixels = image_height - desired_box_height
  from_left_box_start = int(extra_width_pixels / 2)
  right_end = from_left_box_start + desired_box_width
  from_top_box_start = int(extra_height_pixels / 2)
  bottom_end = from_top_box_start + desired_box_height  
  
  box = (from_left_box_start, from_top_box_start, right_end, bottom_end)
  print(f"Box is {box}")
  return box



# for region in region_to_full_pp_cover_paths:
#   for image_path in region_to_full_pp_cover_paths[region]:

# swiss_reisenpass = Image.open("./ppcovers/europe/Switzerland.png")
# swiss_reisenpass.show()
# box = get_box_for_image_path("./ppcovers/europe/Switzerland.png")
# resize = swiss_reisenpass.resize((705, 1000))
# print(f"Region shape is {resize.size}")
# resize.show()

In [5]:
def merge_horizontally(im1, im2):
    w = im1.size[0] + im2.size[0]
    h = max(im1.size[1], im2.size[1])
    im = Image.new("RGBA", (w, h))

    im.paste(im1)
    im.paste(im2, (im1.size[0], 0))

    return im

def merge_vertically(im1, im2, center=False):
    h = im1.size[1] + im2.size[1]
    w = max(im1.size[0], im2.size[0])
    im = Image.new("RGBA", (w, h))

    if center: 
    # xxxxxxxx
    #   xx  
        smaller_image_width = min(im1.size[0], im2.size[0])
        
        wider_image_width - smaller_image_width / 2 
    else:
        im.paste(im1)
        im.paste(im2, (0, im1.size[1]))

    return im

def get_processed_image_from_path(image_path,remove_top_left=False):
    pp_image = Image.open(image_path)
    bottom_crop_length = 25
    bar_width = 5

    box = (bar_width,bar_width,705 - bottom_crop_length, 1000 - bottom_crop_length)
    resized_image = pp_image.resize((705, 1000))
    resized_image = resized_image.crop(box)

    return resized_image    

In [6]:
def get_country_name_from_path(path):
  country_name = path.split("/")[-1][0:-4]
  return country_name

def get_poster(images_per_row,image_paths):

  print(f"Creating poster with {len(image_paths)} pp covers")
  num_rows = len(image_paths) / images_per_row
  print(f"Aspect ratio will be {(2*images_per_row)/(3*num_rows)}")
  row_images = []
  elements_in_row_so_far = 0
  current_row_image = None
  for image_path in image_paths:
    country_name = get_country_name_from_path(image_path)
    cut_top_and_bottom_bar = country_name in set([
      "Argentina","Australia","Chile","Bulgaria","Bosnia+Herzegovina","Bolivia","China","Germany", "Guyana"
    ])
    processed_image = get_processed_image_from_path(image_path,True)
    if current_row_image is None:
      current_row_image = processed_image
      elements_in_row_so_far = 1
    elif elements_in_row_so_far < images_per_row:
      current_row_image = merge_horizontally(current_row_image,processed_image)
      elements_in_row_so_far += 1
    else:
      # start new row
      print(f"Adding new row to list")
      row_images.append(current_row_image)
      current_row_image = processed_image
      elements_in_row_so_far = 1
  
  if current_row_image is not None:
    row_images.append(current_row_image)

  # merge all rows vertically
  poster = None
  print(f"There are {len(row_images)} rows to merge")
  for row_image in row_images:
    if poster is None:
      poster = row_image
    else:
      poster = merge_vertically(poster,row_image)
  return poster

In [7]:
def get_inner_most_path(absolute_path):
  innermost_path = absolute_path.split("/")[-1]
  return innermost_path

all_pp_paths = []
[[all_pp_paths.append(country) for country in countries_per_region] for countries_per_region in list(region_to_full_pp_cover_paths.values())]

# for col_num in range(9,21):
#   poster = get_poster(col_num,sorted(all_pp_paths,key=get_inner_most_path))
#   poster.show()  

[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  Non

In [14]:
  poster = get_poster(23,sorted(all_pp_paths,key=get_inner_most_path))
  poster.show()  

Creating poster with 199 pp covers
Aspect ratio will be 1.7721943048576212
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
Adding new row to list
There are 9 rows to merge


In [9]:
# for col_num in range(2,5):
poster = get_poster(10,sorted(region_to_full_pp_cover_paths["south_america"]))
  # poster.show()  

Creating poster with 13 pp covers
Aspect ratio will be 5.128205128205128
Adding new row to list
There are 2 rows to merge


In [10]:
# for col_num in range(5,11):
#   poster = get_poster(col_num,sorted(region_to_full_pp_cover_paths["north_america"]))
#   poster.show()  

In [11]:
# poster = get_poster(14,sorted(region_to_full_pp_cover_paths["north_america"]))
# poster.show()


In [12]:
# for col_num in range(5,11):
#   poster = get_poster(col_num,sorted(region_to_full_pp_cover_paths["asia"]))
#   poster.show()  

In [13]:
# for col_num in range(5,11):
#   poster = get_poster(col_num,sorted(region_to_full_pp_cover_paths["africa"]))
#   poster.show()  

To figure out next:
1. Passport shape cutout for all images or crop for all
2. Go to market